This code is for the client who is sending all zeros instead of model updates. 

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from create_MNIST_datasets import get_MNIST, plot_samples

In [2]:
import syft as sy

In [3]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!



Each client will launch the duet and from the server side the server would join the duet. When a connection is established, then it would show connected in the drop box and a live status which is shown by a star, would animate.

In [4]:
mnist_iid_train_dls = torch.load('client0.pth')

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = F.softmax(x)
        return x


model_client = CNN()


In [6]:
x = duet.store[0].get()

By duet.get - the client got the initial model parameters and the would now run the code upto the end.

In [7]:
torch.nn.utils.vector_to_parameters(x,model_client.parameters())

In [8]:
model_client_1 = deepcopy(model_client)

# Model Training

In [9]:
def set_to_zero_model_weights(model):


    for layer_weigths in model.parameters():
        layer_weigths.data.sub_(layer_weigths.data)
        

In [10]:
set_to_zero_model_weights(model_client)

In [11]:
parameters = torch.nn.utils.parameters_to_vector(model_client.parameters())

In [12]:
parameters.send(duet)

/Users/syeda/courses/fl/lib/python3.8/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


In [13]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 42a156fc55054551a3ad6f58320be23a>,[],,<class 'torch.Tensor'>


In [14]:
duet.requests.add_handler(action="accept")

[2022-12-03T13:24:39.404628-0500][CRITICAL][logger]][47271] You do not have permission to .get() Object with ID: <UID: 42a156fc55054551a3ad6f58320be23a>Please submit a request.
[2022-12-03T13:24:39.407353-0500][CRITICAL][logger]][47271] You do not have permission to .get() Object with ID: <UID: 42a156fc55054551a3ad6f58320be23a>Please submit a request.


The client has sent the model update by .send(duet) and has accepted the request from the server to access the updated model.

The client can see any updates or any data sent from the server in the live status. Also in the live status the request updates from the server will also be shown. If there is no updates then Objects: 0  Requests: 0   Messages: 0  Request Handlers: 0. And if there is any update on any of them the number would change to 0 or 1 or accordingly. 